# Plastic Optimization

We attempt to design new plastic molecule using LDM.
Our main goal is on
1. Straw
2. ?

## Setup enviroment
!git clone https://github.com/Ahnd6474/KSEF

%cd your_shit/GitHub/KSEF

!pip install -r requirements.txt

In [1]:
import pandas as pd
from __future__ import annotations

import os
import pickle
from pathlib import Path
from typing import Iterable, Iterator, Optional
import pyarrow
import torch
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset, load_model, sampling, visualize_molecule_3d

## Read Data

In [2]:
df=pd.read_parquet('data/plastic.parquet')
df

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2,smiles,num_side,num_back,end_group,names
0,0.0,246.341049,354.051544,524.699646,773.208252,25.838213,32.409279,103.836082,-0.074397,-0.109570,0.493981,-0.071000,-0.061771,0.296051,"[[*]OCCC(=O)[*], [*]OCCC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, ]","[pha, pha]"
1,0.1,313.798401,449.257751,502.616241,1866.776855,44.379429,38.010651,7.215131,-0.043089,-0.006026,0.462577,-0.049233,-0.031425,0.342873,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
2,0.2,306.681305,446.618713,502.605896,1769.267456,44.743877,37.998493,8.727348,-0.050900,-0.028856,0.455602,-0.060321,-0.039934,0.322175,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
3,0.3,298.431488,442.813019,503.664642,1653.973633,44.287182,37.676380,10.997544,-0.057812,-0.048113,0.458959,-0.069126,-0.047438,0.314363,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
4,0.4,290.073944,436.398560,505.013275,1508.705688,42.840149,36.907192,14.239414,-0.065853,-0.065725,0.461121,-0.075789,-0.055182,0.302125,"[[*]OCCC(=O)[*], [*]OC(O)CC(=O)[*]]","[0.0, 0.0]","[3.0, 3.0]","[, O]","[pha, pha]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373498,0.0,325.252380,462.438263,502.465759,2680.520020,39.839355,68.497833,30.515642,0.011635,-0.060768,0.179605,0.002471,0.001415,0.056135,"[[*]CC([*])O, [*]CC([*])O]","[nan, nan]","[nan, nan]","[None, None]","[PVA, PVA]"
1373499,0.0,369.788818,509.966248,587.491455,2227.901367,33.554047,37.865055,1.950041,2.547638,11.337365,21.572718,0.559539,0.881040,24.865118,"[[*]CC([*])c1ccccc1, [*]CC([*])c1ccccc1]","[nan, nan]","[nan, nan]","[None, None]","[PS, PS]"
1373500,0.0,344.375183,457.250885,574.055664,1724.992798,50.262169,46.697941,27.839941,0.099392,0.332474,1.581858,-0.005775,0.070204,1.482780,"[[*]CC([*])Cl, [*]CC([*])Cl]","[nan, nan]","[nan, nan]","[None, None]","[PVC, PVC]"
1373501,0.0,393.653473,536.671326,700.403137,1942.565186,44.805931,65.647507,27.729816,0.016741,0.138303,1.767082,-0.054595,-0.069961,1.336078,"[[*]CCOC(=O)c1ccc2cc(C(=O)O[*])ccc2c1, [*]CCOC...","[nan, nan]","[nan, nan]","[None, None]","[PEN, PEN]"


In [3]:
df.describe()

,comp,Tg,Tm,Td,YM,TS_y,TS_b,eps_b,perm_O2,perm_CO2,perm_He,perm_N2,perm_CH4,perm_H2
count,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06,1.373503e+06
mean,4.997987e-01,2.725839e+02,3.529060e+02,5.769558e+02,5.405507e+02,2.073843e+01,2.460826e+01,2.115185e+02,8.110294e-01,4.332410e+00,7.026700e+00,1.901237e-01,4.661335e-01,7.116223e+00
std,2.583418e-01,2.701165e+01,3.713276e+01,2.686434e+01,4.882776e+02,1.366027e+01,1.229784e+01,1.205617e+02,8.184573e-01,3.972706e+00,4.018037e+00,2.560846e-01,5.097858e-01,4.612499e+00
min,0.000000e+00,2.020853e+02,2.794441e+02,4.246918e+02,9.102757e+01,3.014123e+00,4.558969e+00,4.952888e-01,-9.569907e-02,-1.267213e-01,1.728067e-01,-1.294968e-01,-1.326250e-01,2.026558e-05
25%,3.000000e-01,2.543430e+02,3.266345e+02,5.673138e+02,1.744876e+02,8.976903e+00,1.493889e+01,1.076303e+02,3.791428e-01,1.926220e+00,4.530061e+00,5.799794e-02,1.869081e-01,4.429616e+00
50%,5.000000e-01,2.699055e+02,3.456665e+02,5.827677e+02,3.266322e+02,1.742608e+01,2.180278e+01,2.111784e+02,6.681085e-01,3.541990e+00,6.793644e+00,1.420151e-01,3.605243e-01,6.750260e+00
75%,7.000000e-01,2.883478e+02,3.725729e+02,5.933018e+02,7.870456e+02,3.001149e+01,3.317859e+01,3.043738e+02,1.066986e+00,5.741556e+00,9.030967e+00,2.563907e-01,6.126437e-01,9.138975e+00
max,9.000000e-01,4.033092e+02,5.529254e+02,7.004031e+02,3.081163e+03,7.795893e+01,1.298841e+02,5.666409e+02,3.425747e+01,1.253688e+02,1.188544e+02,8.895615e+00,1.651693e+01,1.671498e+02


## Property Prediction
We use MLP multi regressor to predict property of plastic

### Load Model and helpers

In [9]:
from __future__ import annotations

import ast
import pickle
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

from geoldm import encode, smiles_to_3d
from geoldm.configs import get_dataset_info
from geoldm.qm9 import dataset as qm9_dataset

# -------------------------
# 설정
# -------------------------

DATA_PATH = Path("data/plastic.parquet")               # 입력 데이터
CHECKPOINT_DIR = Path("./qm9_latent2")            # GeoLDM QM9 checkpoint 디렉토리
LATENT_CACHE_PATH = Path("latents.pt")            # SMILES -> latent dict 저장 위치
MODEL_CKPT_PATH = Path("mixture_mlp.pt")          # 학습된 MLP 저장 위치
GRID_CSV_PATH = Path("results1/mixture_grid_search.csv")   # 그리드 서치 결과 CSV

TARGET_COLUMNS = [
    "Tg",
    "Tm",
    "Td",
    "YM",
    "TS_y",
    "TS_b",
    "eps_b",
    "perm_O2",
    "perm_CO2",
    "perm_He",
    "perm_N2",
    "perm_CH4",
    "perm_H2",
]

# grid search에서 사용할 alpha 값들
ALPHA_GRID = [0.1, 0.3, 0.5, 0.7, 0.9]

EPOCHS = 100
EPOCHS2=500
BATCH_SIZE = 128
LR = 1e-3
VALID_RATIO = 0.1
TEST_RATIO = 0.1

# 조성비가 별도 컬럼에 있을 때 그 컬럼명
ALPHA_COLUMN_NAME = "comp"   # 필요하면 여기 바꿔라


# -------------------------
# GeoLDM 로딩 / 3D 변환
# -------------------------

def load_qm9_latent_diffusion(checkpoint_dir: Path):
    """
    GeoLDM latent diffusion 모델 로드 (무조건 CPU).
    """
    args_path = checkpoint_dir / "args.pickle"
    with args_path.open("rb") as handle:
        args = pickle.load(handle)

    # GeoLDM은 CPU로만 돌리게 강제
    setattr(args, "cuda", False)

    dataset_info = get_dataset_info(args.dataset, args.remove_h)
    dataloaders, _ = qm9_dataset.retrieve_dataloaders(args)
    train_loader = dataloaders["train"]

    from geoldm import load_model

    model, nodes_dist, _ = load_model(
        stage="latent_diffusion",
        args=args,
        dataset_info=dataset_info,
        dataloader_train=train_loader,
        checkpoint_path=checkpoint_dir,
    )

    device = torch.device("cpu")
    model.to(device)

    return model, dataset_info, nodes_dist, device


def conformer_to_tensors(
    conformer: Dict,
    dataset_info: Dict,
    device: torch.device,
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor], torch.Tensor, torch.Tensor]:
    """
    RDKit conformer 하나를 GeoLDM 인코더가 먹을 수 있는 텐서로 변환.
    - 좌표는 원점 기준으로 센터링해서 mean-zero 조건 만족.
    - QM9이 모르는 원소(S, Cl 등)는 일단 O로 매핑.
    """
    atom_decoder: List[str] = dataset_info["atom_decoder"]  # ['H','C','N','O','F'] 같은 것
    atom_encoder = {symbol: idx for idx, symbol in enumerate(atom_decoder)}

    mapped_indices: List[int] = []
    for symbol in conformer["atom_symbols"]:
        if symbol not in atom_encoder:
            print(f"[WARN] Unknown atom type {symbol!r} -> mapped to 'O'")
            if "O" in atom_encoder:
                symbol = "O"
            else:
                symbol = atom_decoder[0]
        mapped_indices.append(atom_encoder[symbol])

    atom_indices = torch.tensor(mapped_indices, dtype=torch.long, device=device)
    one_hot = F.one_hot(atom_indices, num_classes=len(atom_decoder)).float()

    positions = torch.tensor(
        conformer["coordinates"], dtype=torch.float32, device=device
    )
    positions = positions - positions.mean(dim=0, keepdim=True)

    x = positions.unsqueeze(0)  # (1, N, 3)
    h = {
        "categorical": one_hot.unsqueeze(0),  # (1, N, C)
        "integer": torch.zeros(one_hot.shape[0], 1, device=device).unsqueeze(0),
    }

    node_mask = torch.ones(x.shape[0], x.shape[1], 1, device=device)
    edge_mask = node_mask.squeeze(-1)[..., None] * node_mask.squeeze(-1)[:, None]
    return x, h, node_mask, edge_mask


def clean_bigsmiles(big: str) -> str:
    """
    BigSMILES의 [*] anchor를 간단히 제거.
    필요하면 여기서 더 복잡하게 전처리해도 됨.
    """
    return big.replace("[*]", "")


def build_latent_cache():
    """
    plastic.parquet에서 고유 분자들(BigSMILES)을 뽑아서
    GeoLDM encoder로 latent 풀링벡터를 만든 뒤
    {원본 BigSMILES: pooled_latent} dict를 LATENT_CACHE_PATH에 저장.
    """
    print(">> plastic.parquet 로드 중...")
    df = pd.read_parquet(DATA_PATH)

    # 1) 고유 분자 문자열 수집
    unique_mols = set()
    for raw in df["smiles"]:
        item = raw
        if isinstance(item, str):
            item = ast.literal_eval(item)
        elif isinstance(item, np.ndarray):
            item = item.tolist()

        if not isinstance(item, (list, tuple)) or len(item) < 2:
            raise ValueError(f"예상과 다른 형태의 값: {repr(item)}")

        mol1, mol2 = item[0], item[1]
        unique_mols.add(mol1)
        unique_mols.add(mol2)

    unique_mols = sorted(unique_mols)
    print(f">> 고유 분자 수: {len(unique_mols)}")

    # 2) GeoLDM 로드 (CPU)
    print(">> GeoLDM latent diffusion 모델 로드 중 (CPU)...")
    model, dataset_info, nodes_dist, device = load_qm9_latent_diffusion(CHECKPOINT_DIR)
    model.eval()

    latent_dict: Dict[str, torch.Tensor] = {}

    # 3) 각 분자에 대해 BigSMILES -> (anchor 제거) -> 3D -> latent
    print(">> 각 분자 latent 계산 중...")
    with torch.no_grad():
        for big_smi in tqdm(unique_mols, desc="Encoding molecules"):
            cleaned = clean_bigsmiles(big_smi)
            try:
                conformers = smiles_to_3d(cleaned)
            except Exception as e:
                print(f"[WARN] 3D 생성 실패: {big_smi} ({e}), skip")
                continue

            if len(conformers) == 0:
                print(f"[WARN] 3D conformer 없음: {big_smi}, skip")
                continue

            conformer = conformers[0]
            x, h, node_mask, edge_mask = conformer_to_tensors(
                conformer, dataset_info, device
            )

            # GeoLDM latent encoder
            z_x, z_sigma_x, z_h, z_sigma_h = encode(
                model, x, h, node_mask=node_mask, edge_mask=edge_mask
            )
            # 노드 차원 mean pooling 후 concat
            z_x_mean = z_x.mean(dim=1).squeeze(0)  # (Dx,)
            z_h_mean = z_h.mean(dim=1).squeeze(0)  # (Dh,)
            pooled = torch.cat([z_x_mean, z_h_mean], dim=-1).cpu()

            latent_dict[big_smi] = pooled

    print(">> latent dict 저장 중:", LATENT_CACHE_PATH)
    torch.save(latent_dict, LATENT_CACHE_PATH)
    print(">> latent dict 저장 완료.")


# -------------------------
# Dataset / MLP
# -------------------------

class MixtureDataset(Dataset):
    """
    한 샘플: (z_A, z_B, alpha) -> feature 벡터, target은 scaled property.
    alpha는
      - 우선 df[ALPHA_COLUMN_NAME]에서 찾고,
      - 없으면 smiles 리스트의 3번째 원소(item[2])에서 찾는다.
    """

    def __init__(
        self,
        df: pd.DataFrame,
        latent_dict: Dict[str, torch.Tensor],
        target_means: np.ndarray,
        target_stds: np.ndarray,
    ):
        self.rows = []
        self.latent_dict = latent_dict

        has_alpha_col = ALPHA_COLUMN_NAME in df.columns

        for _, row in df.iterrows():
            raw = row["smiles"]
            item = raw
            if isinstance(item, str):
                item = ast.literal_eval(item)
            elif isinstance(item, np.ndarray):
                item = item.tolist()

            if not isinstance(item, (list, tuple)) or len(item) < 2:
                # 최소한 분자 2개는 있어야 한다
                continue

            mol1, mol2 = item[0], item[1]

            # alpha 추출
            if has_alpha_col:
                alpha = float(row[ALPHA_COLUMN_NAME])
            elif len(item) >= 3:
                alpha = float(item[2])
            else:
                # 조성비를 어디서도 못 찾으면 스킵
                continue

            # latent 없는 분자는 스킵
            if mol1 not in latent_dict or mol2 not in latent_dict:
                continue

            y = row[TARGET_COLUMNS].values.astype(np.float32)
            self.rows.append((mol1, mol2, alpha, y))

        self.target_means = target_means.astype(np.float32)
        self.target_stds = target_stds.astype(np.float32)

        if len(self.rows) > 0:
            some_latent = next(iter(latent_dict.values()))
            self.latent_dim = some_latent.numel()
        else:
            self.latent_dim = 0

    def __len__(self) -> int:
        return len(self.rows)

    def __getitem__(self, idx: int):
        mol1, mol2, alpha, y = self.rows[idx]
        z_A = self.latent_dict[mol1].float()
        z_B = self.latent_dict[mol2].float()
        alpha = float(alpha)

        # mixture feature
        h_mix = alpha * z_A + (1.0 - alpha) * z_B
        h_int = torch.abs(z_A - z_B)
        alpha_tensor = torch.tensor([alpha], dtype=torch.float32)

        # [z_A, z_B, h_mix, h_int, alpha]
        x = torch.cat([z_A, z_B, h_mix, h_int, alpha_tensor], dim=0)

        # target scaling
        y_scaled = (y - self.target_means) / self.target_stds
        y_scaled = torch.from_numpy(y_scaled)

        return x, y_scaled


class MixtureMLP(nn.Module):
    """
    2~3층 정도의 단순 MLP. 여기서는 512, 256 두 층 사용.
    """

    def __init__(self, input_dim: int, output_dim: int, hidden_sizes=(512, 256)):
        super().__init__()
        layers: List[nn.Module] = []
        prev = input_dim
        for h in hidden_sizes:
            layers += [nn.Linear(prev, h), nn.ReLU()]
            prev = h
        layers.append(nn.Linear(prev, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


# -------------------------
# 학습 / 평가 / 그리드 서치
# -------------------------

def compute_target_stats(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """
    TARGET_COLUMNS에 대한 mean / std 계산.
    """
    y = df[TARGET_COLUMNS].values.astype(np.float32)
    means = y.mean(axis=0)
    stds = y.std(axis=0) + 1e-8  # 0 방지
    return means, stds


def train_mixture_mlp():
    """
    기존 scaled MSE로 학습된 모델이 있으면 그 체크포인트에서 이어서
    Kendall-style uncertainty-weighted loss로 파인튜닝한다.
    """
    print(">> plastic.parquet 로드 중...")
    df = pd.read_parquet(DATA_PATH)

    print(">> latent dict 로드 중...")
    latent_dict: Dict[str, torch.Tensor] = torch.load(LATENT_CACHE_PATH)

    # -------------------------
    # 1) 타깃 스케일링 통계 + ckpt 로드 (있으면 이어서)
    # -------------------------
    ckpt = None
    if MODEL_CKPT_PATH.exists():
        print(">> 기존 체크포인트 발견: scaled MSE로 학습된 모델에서 이어서 학습합니다.")
        ckpt = torch.load(
        MODEL_CKPT_PATH,
        map_location="cpu",
        weights_only=False,  # <<< 추가
        )
        target_means = ckpt["target_means"]
        target_stds = ckpt["target_stds"]
        ckpt_input_dim = ckpt["input_dim"]
        output_dim = ckpt["output_dim"]
    else:
        print(">> 기존 체크포인트 없음: 새로 target 통계 계산 후 처음부터 학습합니다.")
        target_means, target_stds = compute_target_stats(df)
        ckpt_input_dim = None
        output_dim = len(TARGET_COLUMNS)

    print(">> target means:", target_means)
    print(">> target stds:", target_stds)

    # -------------------------
    # 2) Dataset 구성
    # -------------------------
    dataset = MixtureDataset(df, latent_dict, target_means, target_stds)
    n_total = len(dataset)
    print(">> 전체 유효 샘플 수:", n_total)

    if n_total == 0:
        raise RuntimeError(
            "MixtureDataset이 비어 있음. "
            "smiles에 [mol1, mol2] + comp(조성비) 구조가 맞는지, "
            "GeoLDM latent가 제대로 만들어졌는지 확인해라."
        )

    n_valid = int(n_total * VALID_RATIO)
    n_test = int(n_total * TEST_RATIO)
    n_train = n_total - n_valid - n_test

    train_dataset, valid_dataset, test_dataset = random_split(
        dataset, [n_train, n_valid, n_test],
        generator=torch.Generator().manual_seed(42),
    )

    input_dim = dataset[0][0].numel()
    print(">> input_dim:", input_dim, "output_dim:", output_dim)

    if ckpt_input_dim is not None and input_dim != ckpt_input_dim:
        raise RuntimeError(
            f"입력 차원이 기존 ckpt({ckpt_input_dim})와 다름: {input_dim}"
        )

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # -------------------------
    # 3) 모델 + log_vars 세팅 (이어붙이기)
    # -------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MixtureMLP(input_dim=input_dim, output_dim=output_dim, hidden_sizes=(512, 256))
    model.to(device)

    # Kendall et al. uncertainty weighting용 파라미터: 길이 T = output_dim
    log_vars = nn.Parameter(torch.zeros(output_dim, device=device))

    # 기존 ckpt가 있으면 가중치 로드 (scaled MSE로 학습된 상태에서 시작)
    if ckpt is not None:
        model.load_state_dict(ckpt["model_state_dict"])
        # 만약 이후에 또 이어붙이기 할 때를 대비해 ckpt에 log_vars가 있으면 복원
        if "log_vars" in ckpt:
            with torch.no_grad():
                log_vars.copy_(ckpt["log_vars"].to(device))
        print(">> 기존 모델 가중치 로드 완료.")

    # 모델 파라미터 + log_vars를 같이 최적화
    optimizer = torch.optim.Adam(
        list(model.parameters()) + [log_vars],
        lr=LR,
    )

    writer = SummaryWriter(log_dir="runs/polymer_mixture_uncertainty")

    best_valid_loss = float("inf")
    global_step = 0

    # -------------------------
    # 4) Training loop (Kendall loss)
    # -------------------------
    for epoch in range(1, EPOCHS2 + 1):
        model.train()
        train_loss = 0.0
        n_train_batches = 0

        for x_batch, y_batch in tqdm(
            train_loader, desc=f"Train epoch {epoch}", leave=False
        ):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(x_batch)         # (B, T)
            err = y_pred - y_batch          # (B, T)

            # 태스크별 MSE (배치 평균, scaled space)
            mse_per_task = (err ** 2).mean(dim=0)   # (T,)

            # Kendall et al. 2018 스타일:
            # L = 0.5 * Σ_t [ exp(-s_t) * MSE_t + s_t ],  s_t = log_vars[t]
            loss = 0.5 * torch.sum(torch.exp(-log_vars) * mse_per_task + log_vars)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            n_train_batches += 1

            writer.add_scalar("train/batch_loss", loss.item(), global_step)
            global_step += 1

        train_loss /= max(n_train_batches, 1)

        # log_vars 모니터링
        for i, name in enumerate(TARGET_COLUMNS):
            writer.add_scalar(f"train/log_var_{name}", log_vars[i].item(), epoch)

        # -------------------------
        # 5) Validation (같은 loss 정의)
        # -------------------------
        model.eval()
        valid_loss = 0.0
        n_valid_batches = 0
        per_target_sqerr = np.zeros(output_dim, dtype=np.float64)
        n_valid_samples = 0

        with torch.no_grad():
            for x_batch, y_batch in tqdm(
                valid_loader, desc=f"Valid epoch {epoch}", leave=False
            ):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                y_pred = model(x_batch)
                err = y_pred - y_batch
                mse_per_task = (err ** 2).mean(dim=0)   # (T,)

                val_loss_tensor = 0.5 * torch.sum(torch.exp(-log_vars) * mse_per_task + log_vars)
                loss_val = val_loss_tensor

                valid_loss += loss_val.item()
                n_valid_batches += 1

                # 타깃별 MSE 통계용 (scaled space)
                err_np = err.cpu().numpy()
                per_target_sqerr += (err_np ** 2).sum(axis=0)
                n_valid_samples += err_np.shape[0]

        valid_loss /= max(n_valid_batches, 1)
        per_target_mse_scaled = per_target_sqerr / max(n_valid_samples, 1)
        per_target_mse_original = (target_stds ** 2) * per_target_mse_scaled

        print(
            f"[Epoch {epoch}] train_loss={train_loss:.4f}, "
            f"valid_loss={valid_loss:.4f}"
        )

        writer.add_scalar("train/epoch_loss", train_loss, epoch)
        writer.add_scalar("valid/epoch_loss", valid_loss, epoch)

        for i, name in enumerate(TARGET_COLUMNS):
            writer.add_scalar(f"valid/MSE_scaled_{name}", per_target_mse_scaled[i], epoch)
            writer.add_scalar(f"valid/MSE_original_{name}", per_target_mse_original[i], epoch)

        # best 모델 저장 (이제는 log_vars도 같이 저장)
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            print(f"  >> 베스트 모델 갱신 (valid_loss={valid_loss:.4f})")
            torch.save(
                {
                    "model_state_dict": model.state_dict(),
                    "input_dim": input_dim,
                    "output_dim": output_dim,
                    "target_means": target_means,
                    "target_stds": target_stds,
                    "log_vars": log_vars.detach().cpu(),
                },
                MODEL_CKPT_PATH,
            )

    writer.close()

    # -------------------------
    # 6) Test 평가 (per-target MSE, 원래 스케일)
    # -------------------------
    print(">> 최종 모델 로드 후 test set 평가")
    ckpt_final = torch.load(
    MODEL_CKPT_PATH,
    map_location=device,
    weights_only=False,  # <<< 추가
    )

    model.load_state_dict(ckpt_final["model_state_dict"])
    model.to(device)
    model.eval()

    per_target_sqerr = np.zeros(output_dim, dtype=np.float64)
    n_test_samples = 0

    with torch.no_grad():
        for x_batch, y_batch in tqdm(test_loader, desc="Test", leave=False):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            y_pred = model(x_batch)
            err = (y_pred - y_batch).cpu().numpy()
            per_target_sqerr += (err ** 2).sum(axis=0)
            n_test_samples += err.shape[0]

    per_target_mse_scaled = per_target_sqerr / max(n_test_samples, 1)
    per_target_mse_original = (target_stds ** 2) * per_target_mse_scaled

    print(">> Test MSE (원래 스케일 기준):")
    for i, name in enumerate(TARGET_COLUMNS):
        print(f"  {name}: {per_target_mse_original[i]:.4f}")



def grid_search_all_pairs():
    """
    latent_dict에 들어있는 모든 분자 조합(i <= j)과
    ALPHA_GRID의 alpha 값에 대해 MLP 예측을 수행하고
    결과를 CSV로 저장한다.
    """
    print(">> latent dict 로드 중...")
    latent_dict: Dict[str, torch.Tensor] = torch.load(LATENT_CACHE_PATH)
    mol_list = sorted(latent_dict.keys())
    N = len(mol_list)
    print(">> 분자 수:", N)

    print(">> MLP 체크포인트 로드 중...")
    ckpt = torch.load(
    MODEL_CKPT_PATH,
    map_location="cpu",
    weights_only=False,  # <<< 추가
    )

    input_dim = ckpt["input_dim"]
    output_dim = ckpt["output_dim"]
    target_means = ckpt["target_means"]
    target_stds = ckpt["target_stds"]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MixtureMLP(input_dim=input_dim, output_dim=output_dim, hidden_sizes=(512, 256))
    model.load_state_dict(ckpt["model_state_dict"])
    model.to(device)
    model.eval()

    rows_out = []

    total_pairs = N * (N + 1) // 2
    print(">> 모든 분자 조합 × alpha grid 예측 중...")

    with torch.no_grad():
        with tqdm(total=total_pairs, desc="Pairs") as pbar:
            for i in range(N):
                mol1 = mol_list[i]
                z_A_full = latent_dict[mol1].to(device).float()
                for j in range(i, N):
                    mol2 = mol_list[j]
                    z_B_full = latent_dict[mol2].to(device).float()

                    features = []
                    alphas = []
                    for alpha in ALPHA_GRID:
                        alpha = float(alpha)
                        h_mix = alpha * z_A_full + (1.0 - alpha) * z_B_full
                        h_int = torch.abs(z_A_full - z_B_full)
                        alpha_tensor = torch.tensor([alpha], dtype=torch.float32, device=device)
                        x = torch.cat(
                            [z_A_full, z_B_full, h_mix, h_int, alpha_tensor], dim=0
                        )
                        features.append(x)
                        alphas.append(alpha)

                    x_batch = torch.stack(features, dim=0)
                    y_pred_scaled = model(x_batch)
                    y_pred_scaled_np = y_pred_scaled.cpu().numpy()
                    y_pred_original = y_pred_scaled_np * target_stds + target_means

                    for k, alpha in enumerate(alphas):
                        row = {
                            "smiles_A": mol1,
                            "smiles_B": mol2,
                            "alpha": alpha,
                        }
                        for t_idx, name in enumerate(TARGET_COLUMNS):
                            row[name] = float(y_pred_original[k, t_idx])
                        rows_out.append(row)

                    pbar.update(1)

    print(">> DataFrame으로 변환 및 CSV 저장 중...")
    out_df = pd.DataFrame(rows_out)
    out_df.to_csv(GRID_CSV_PATH, index=False)
    print(">> CSV 저장 완료:", GRID_CSV_PATH)


# -------------------------
# main
# -------------------------

if __name__ == "__main__":
    # 1) 처음 한 번만 latent 캐시 생성
    if not LATENT_CACHE_PATH.exists():
        build_latent_cache()

    # 2) MLP 학습
    train_mixture_mlp()

    # 3) 전체 조합 × alpha grid 서치
    grid_search_all_pairs()


>> plastic.parquet 로드 중...
>> latent dict 로드 중...
>> 기존 체크포인트 발견: scaled MSE로 학습된 모델에서 이어서 학습합니다.
>> target means: [2.7258392e+02 3.5290601e+02 5.7695581e+02 5.4055072e+02 2.0738430e+01
 2.4608255e+01 2.1151849e+02 8.1102943e-01 4.3324103e+00 7.0267005e+00
 1.9012366e-01 4.6613353e-01 7.1162233e+00]
>> target stds: [2.7011644e+01 3.7132740e+01 2.6864326e+01 4.8827740e+02 1.3660261e+01
 1.2297838e+01 1.2056164e+02 8.1845707e-01 3.9727042e+00 4.0180354e+00
 2.5608444e-01 5.0978565e-01 4.6124973e+00]
>> 전체 유효 샘플 수: 1334615
>> input_dim: 21 output_dim: 13
>> 기존 모델 가중치 로드 완료.


Train epoch 1:   0%|          | 0/8342 [00:00<?, ?it/s]

Valid epoch 1:   0%|          | 0/1043 [00:00<?, ?it/s]

[Epoch 1] train_loss=-1.4419, valid_loss=-1.4354
  >> 베스트 모델 갱신 (valid_loss=-1.4354)


Train epoch 2:   0%|          | 0/8342 [00:00<?, ?it/s]

...

[Epoch 500] train_loss=-2.6352, valid_loss=-2.6063
>> 최종 모델 로드 후 test set 평가


Test:   0%|          | 0/1043 [00:00<?, ?it/s]

>> Test MSE (원래 스케일 기준):
  Tg: 215.2875
  Tm: 393.1185
  Td: 280.9041
  YM: 75073.9477
  TS_y: 59.3599
  TS_b: 53.5339
  eps_b: 6438.1253
  perm_O2: 0.0756
  perm_CO2: 2.9189
  perm_He: 3.6527
  perm_N2: 0.0094
  perm_CH4: 0.0372
  perm_H2: 4.0112
>> latent dict 로드 중...
>> 분자 수: 545
>> MLP 체크포인트 로드 중...
>> 모든 분자 조합 × alpha grid 예측 중...


Pairs:   0%|          | 0/148785 [00:00<?, ?it/s]

>> DataFrame으로 변환 및 CSV 저장 중...
>> CSV 저장 완료: mixture_grid_search.csv


In [10]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 이 코드는 다음이 이미 정의되어 있다고 가정:
# - DATA_PATH (Path or str, e.g. Path("plastic.parquet"))
# - LATENT_CACHE_PATH (Path to "latents.pt")
# - MODEL_CKPT_PATH (Path to "mixture_mlp.pt")
# - TARGET_COLUMNS (list of 14 target names)
# - MixtureDataset, MixtureMLP
#
# 만약 다른 파일에 있다면:
# from your_module import DATA_PATH, LATENT_CACHE_PATH, MODEL_CKPT_PATH, TARGET_COLUMNS, MixtureDataset, MixtureMLP

def compute_target_stats(df: pd.DataFrame):
    y = df[TARGET_COLUMNS].values.astype(np.float32)
    means = y.mean(axis=0)
    stds = y.std(axis=0) + 1e-8
    return means, stds


def visualize_predictions_scatter_with_std():
    print(">> Loading data and latent dict...")
    df = pd.read_parquet(DATA_PATH)
    latent_dict = torch.load(LATENT_CACHE_PATH)

    # training 때와 동일한 target_means/target_stds 계산
    target_means, target_stds = compute_target_stats(df)

    # 전체 데이터셋 대상으로 Dataset 구성
    full_dataset = MixtureDataset(df, latent_dict, target_means, target_stds)
    full_loader = DataLoader(full_dataset, batch_size=512, shuffle=False)

    # 모델 로드
    print(">> Loading trained model...")
    ckpt = torch.load(MODEL_CKPT_PATH, map_location="cpu", weights_only=False)
    input_dim = ckpt["input_dim"]
    output_dim = ckpt["output_dim"]
    assert output_dim == len(TARGET_COLUMNS)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MixtureMLP(input_dim=input_dim, output_dim=output_dim, hidden_sizes=(512, 256))
    model.load_state_dict(ckpt["model_state_dict"])
    model.to(device)
    model.eval()

    # 예측 수집 (scaled space → original space 복원)
    print(">> Computing predictions for entire dataset...")
    all_y_true_scaled = []
    all_y_pred_scaled = []

    with torch.no_grad():
        for x_batch, y_batch_scaled in full_loader:
            x_batch = x_batch.to(device)
            y_batch_scaled = y_batch_scaled.to(device)

            y_pred_scaled = model(x_batch)

            all_y_true_scaled.append(y_batch_scaled.cpu().numpy())
            all_y_pred_scaled.append(y_pred_scaled.cpu().numpy())

    all_y_true_scaled = np.concatenate(all_y_true_scaled, axis=0)  # (N, T)
    all_y_pred_scaled = np.concatenate(all_y_pred_scaled, axis=0)  # (N, T)

    # 스케일 복원
    target_means_np = np.array(target_means, dtype=np.float32)    # (T,)
    target_stds_np = np.array(target_stds, dtype=np.float32)      # (T,)

    all_y_true = all_y_true_scaled * target_stds_np + target_means_np
    all_y_pred = all_y_pred_scaled * target_stds_np + target_means_np

    print(">> Plotting per-target scatter with ±std bands...")

    for t_idx, name in enumerate(TARGET_COLUMNS):
        y_true = all_y_true[:, t_idx]
        y_pred = all_y_pred[:, t_idx]
        std_t = target_stds_np[t_idx]

        # 산점도 범위 설정
        min_val = min(y_true.min(), y_pred.min())
        max_val = max(y_true.max(), y_pred.max())
        margin = 0.05 * (max_val - min_val + 1e-8)
        x_min = min_val - margin
        x_max = max_val + margin

        xs = np.linspace(x_min, x_max, 200)

        plt.figure(figsize=(6, 6))
        plt.scatter(y_true, y_pred, alpha=0.4, s=10)
        # y = x (완벽한 예측)
        plt.plot(xs, xs, linestyle="--", linewidth=1.0, label="y = x")

        # y = x ± std_t (std 기준 양익)
        plt.plot(xs, xs + std_t, linestyle=":", linewidth=1.0, label="y = x + std")
        plt.plot(xs, xs - std_t, linestyle=":", linewidth=1.0, label="y = x - std")

        plt.xlabel(f"True {name}")
        plt.ylabel(f"Predicted {name}")
        plt.title(f"{name}: True vs Predicted (±1 std band)")
        plt.xlim(x_min, x_max)
        plt.ylim(x_min, x_max)
        plt.legend()
        plt.tight_layout()

        out_name = f"scatter_{name}.png"
        plt.savefig(out_name, dpi=200)
        plt.close()
        print(f"  saved {out_name}")

    print(">> Done.")


# 함수 호출
if __name__ == "__main__":
    visualize_predictions_scatter_with_std()


>> Loading data and latent dict...
>> Loading trained model...
>> Computing predictions for entire dataset...
>> Plotting per-target scatter with ±std bands...
  saved scatter_Tg.png
  saved scatter_Tm.png
  saved scatter_Td.png
  saved scatter_YM.png
  saved scatter_TS_y.png
  saved scatter_TS_b.png
  saved scatter_eps_b.png
  saved scatter_perm_O2.png
  saved scatter_perm_CO2.png
  saved scatter_perm_He.png
  saved scatter_perm_N2.png
  saved scatter_perm_CH4.png
  saved scatter_perm_H2.png
>> Done.


[예측값 정의]

logP_CO2  = log(P_CO2)
logP_CH4  = log(P_CH4)
logP_N2   = log(P_N2)

Tg        = 유리전이온도
Td        = 열분해온도
YM        = Young's modulus
TSb       = 인장강도 (stress at break)
epsb      = 연신율 (strain at break)

z1, z2    = 두 모노머의 latent vector


[1. 분리 성능 보상항 J_sep]

J_sep =
    w_perm      * logP_CO2
  + w_sel_CH4   * (logP_CO2 - logP_CH4)   # CO2/CH4 선택성 (log S_CO2/CH4)
  + w_sel_N2    * (logP_CO2 - logP_N2)    # CO2/N2  선택성 (log S_CO2/N2)


[2. 고온·고압용 물성 soft penalty]

임계값(조건)을 다음처럼 둔다 (예시는 너가 quantile 보고 채우면 됨):

Tg_req     = 원하는 최소 Tg
Td_req     = 원하는 최소 Td
YM_min     = 원하는 최소 YM
TSb_min    = 원하는 최소 TSb
epsb_min   = 원하는 최소 epsb

각 penalty는 "부족한 만큼만 제곱"으로 정의:

pen_Tg  = max(0, Tg_req  - Tg)^2
pen_Td  = max(0, Td_req  - Td)^2
pen_YM  = max(0, YM_min  - YM)^2
pen_TS  = max(0, TSb_min - TSb)^2
pen_eps = max(0, epsb_min - epsb)^2


[3. latent prior penalty]

pen_z = ||z1||^2 + ||z2||^2
# (local search면 ||z1 - z1_0||^2 + ||z2 - z2_0||^2 로 바꿔도 됨)


[4. 전체 목적함수 J]

J =
    J_sep
  - λ_Tg  * pen_Tg
  - λ_Td  * pen_Td
  - λ_YM  * pen_YM
  - λ_TS  * pen_TS
  - λ_eps * pen_eps
  - λ_z   * pen_z


[5. 최종 최적화에 쓰는 loss]

loss = -J
